In [ ]:
import os
import re
import glob
import shutil
import subprocess
import nibabel as nib
from nipype.interfaces.dcm2nii import Dcm2niix
from fsl.wrappers import bet, flirt, fslmaths
import numpy as np
import matplotlib
from scipy.ndimage import zoom
import SimpleITK as sitk
from scipy.optimize import least_squares
import matplotlib.pyplot as plt

In [ ]:
source_directory = r'./32ch-shim/32ch_shim_20250306-221509-9362_221533'
destination_directory = r'./32ch_coil_profile'
keys = ['b0ch0','b0ch1','b0ch2','b0ch3','b1ch0','b1ch1','b1ch2','b1ch3',
        'b1ch4','b1ch5','b1ch6','b1ch7','b2ch0','b2ch1','b2ch2','b2ch3',
        'b2ch4','b2ch5','b2ch6','b2ch7','b3ch0','b3ch1','b3ch2','b3ch3',
        'b3ch4','b3ch5','b3ch6','b3ch7','b4ch0','b4ch1','b4ch2','b4ch3']
ch_names =['ch1','ch2','ch3','ch4','ch5','ch6','ch7','ch8',
           'ch9','ch10','ch11','ch12','ch13','ch14','ch15','ch16',
           'ch17','ch18','ch19','ch20','ch21','ch22','ch23','ch24',
           'ch25','ch26','ch27','ch28','ch29','ch30','ch31','ch32',]
rename_dict = dict(zip(keys, ch_names))
pattern = re.compile(r"(" + "|".join(keys) + r")")

def rename_and_move_folders(source_directory, destination_directory):
    """ 遍历 source_directory，将匹配 bXchY 的文件夹重命名为 chX，并进一步分类 0.5n/0.5p """
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    for folder in os.listdir(source_directory):
        folder_path = os.path.join(source_directory, folder)
        
        if os.path.isdir(folder_path): 
            match = pattern.search(folder)
            if match:
                key = match.group(1) 
                new_name = rename_dict[key] 
                
                if "0.5n" in folder:
                    current_dir = "negative"
                elif "0.5p" in folder:
                    current_dir = "positive"
                else:
                    current_dir = "unknown"

                new_folder_path = os.path.join(destination_directory, new_name, current_dir, folder)

                os.makedirs(os.path.dirname(new_folder_path), exist_ok=True)
                
                shutil.move(folder_path, new_folder_path)
                print(f"Moved {folder} -> {new_folder_path}")

rename_and_move_folders(source_directory, destination_directory)


In [ ]:
def dcm2nii(dicom_directory, nifti_folder):
    """
    Converts DICOM files in each subdirectory within `dicom_directory` to NIfTI format and saves them in `nifti_folder`.

    Parameters:
        dicom_directory (str): Path to the directory containing DICOM subdirectories.
        nifti_folder (str): Path to the directory where NIfTI files will be saved.
    """
    directories = os.listdir(dicom_directory)
    
    for directory in directories:
        dicom_file_path = os.path.join(dicom_directory, directory)
        nifti_file_path = os.path.join(nifti_folder, f"{directory}_nii")
        
        converter = Dcm2niix()
        converter.inputs.source_dir = dicom_file_path
        converter.inputs.output_dir = nifti_file_path
    
        try:
            converter.run()
            print(f"Successfully converted {directory} to NIfTI format.")
        except Exception as e:
            print(f"Error converting {directory} to NIfTI: {e}")


In [ ]:
def process_dicom_to_deltaB0(dicom_directory, nifti_folder, delta_TE, Frequency):
    """
    Converts DICOM files to NIfTI format, performs phase unwrapping, applies a brain mask, and calculates delta B0 maps.

    Parameters:
        dicom_directory (str): Path to the directory containing DICOM files.
        nifti_folder (str): Path to the directory where NIfTI files will be saved.
        delta_TE (float): Echo time difference in seconds.
        Frequency (float): Scanner frequency in MHz (e.g., 127.74 MHz for a 3T scanner).
    """
    print("启动！")
    directories = os.listdir(dicom_directory)

    for directory in directories:
        try:
            dicom_file_path = os.path.join(dicom_directory, directory)
            nifti_file_path = os.path.join(nifti_folder, f"{directory}_nii")
            os.makedirs(nifti_file_path, exist_ok=True)

            # Convert DICOM to NIfTI format
            converter = Dcm2niix()
            converter.inputs.source_dir = dicom_file_path
            converter.inputs.output_dir = nifti_file_path
            converter.run()
            # Identify phase and magnitude images
            phase_map = glob.glob(os.path.join(nifti_file_path, "*ph.nii.gz*"))
            magnitude_map = glob.glob(os.path.join(nifti_file_path, "*e2.nii.gz*"))

            if phase_map and magnitude_map:
                # Load the phase image and normalize it to range [-π, π]
                phase_map_nifti = nib.load(phase_map[0])
                phase_map_data = phase_map_nifti.get_fdata()
                affine = phase_map_nifti.affine.copy()
                header = phase_map_nifti.header.copy()

                phasemap = phase_map_data / 4096 * np.pi
                phase_map_img_valid = nib.Nifti1Image(phasemap, affine, header)
                phase_map_img_valid_path = os.path.join(nifti_file_path, f"{directory}_PhaseMap.nii.gz")
                nib.save(phase_map_img_valid, phase_map_img_valid_path)

                # Generate brain mask using BET (Brain Extraction Tool)
                bet_path = os.path.join(nifti_file_path, f"{directory}_bet.nii.gz")
                bet(magnitude_map[0], bet_path, f=0.1, m=True)

                # Apply the brain mask to the phase image
                mask_path = os.path.join(nifti_file_path, f"{directory}_bet_mask.nii.gz")
                brain_map_path = os.path.join(nifti_file_path, f"{directory}_BrainMap.nii.gz")
                fslmaths(phase_map_img_valid_path).mul(mask_path).run(brain_map_path)

                # Perform phase unwrapping
                unwrap_path = os.path.join(nifti_file_path, f"{directory}_unwrapped_Brain.nii.gz")
                cmd = [
                    "prelude",
                    "-p", brain_map_path,
                    "-a", magnitude_map[0],
                    "-o", unwrap_path
                ]
                subprocess.run(cmd, check=True)

                # Calculate delta B0 in Hz
                deltB0_hertz_path = os.path.join(nifti_file_path, f"{directory}_deltaB0_hertz.nii.gz")
                fslmaths(unwrap_path).div(2).div(np.pi).div(delta_TE).run(deltB0_hertz_path)

                # Calculate delta B0 in ppm
                deltB0_ppm_path = os.path.join(nifti_file_path, f"{directory}_deltaB0_ppm.nii.gz")
                fslmaths(deltB0_hertz_path).div(Frequency).run(deltB0_ppm_path)

                print(f"Processing completed for {directory}")
            else:
                print(f"Phase or magnitude images not found for {directory}")
        
        except Exception as e:
            print(f"Error processing {directory}: {e}")

In [ ]:
# Paths for input DICOM and output NIfTI folders
root_directory = r'./32ch_coil_profile'
nifti_folder = r'./32ch_coil_profile_nii'

# Parameters for delta B0 calculation
delta_TE = 0.00246  # Echo time difference in seconds
Frequency = 42.58 * 5  # Scanner frequency in MHz for 5T
for dicom_directory in os.listdir(root_directory):
    ch_dicom_path = os.path.join(root_directory, dicom_directory)
    ch_nii_path = os.path.join(nifti_folder, f"{dicom_directory}_nii")
# Run the main processing and registration functions
    process_dicom_to_deltaB0(ch_dicom_path, ch_nii_path, delta_TE, Frequency)

In [ ]:
for ch_nii in os.listdir(nifti_folder):
    ch_nii_path = os.path.join(nifti_folder, ch_nii)
    neg_path = os.path.join(ch_nii_path, "negative_nii")
    pos_path = os.path.join(ch_nii_path, "positive_nii")
    print(neg_path, pos_path)
    if os.path.exists(neg_path) and os.path.exists(pos_path):
        neg_hertz_fieldmap = [f for f in os.listdir(neg_path) if f.endswith("_hertz.nii.gz")]
        pos_hertz_fieldmap = [f for f in os.listdir(pos_path) if f.endswith("_hertz.nii.gz")]

        for neg_file, pos_file in zip(sorted(neg_hertz_fieldmap), sorted(pos_hertz_fieldmap)):
            neg_img = nib.load(os.path.join(neg_path, neg_file))
            pos_img = nib.load(os.path.join(pos_path, pos_file))

            neg_data = neg_img.get_fdata()
            pos_data = pos_img.get_fdata()

            delta_B0_fieldmap = pos_data - neg_data

            delta_img = nib.Nifti2Image(delta_B0_fieldmap, affine=neg_img.affine)
            delta_path = os.path.join(ch_nii_path,f"{ch_nii}_deltaFieldmap")

            nib.save(delta_img, delta_path)
            print(f"Save: {delta_path}")

In [ ]:
infant_dicom = r"./dicom_infant"
infant_nii = r"./nii_infant"
process_dicom_to_deltaB0(infant_dicom, infant_nii, delta_TE, Frequency)